# 2: Test case: changing the chemistry


<div class="alert alert-info">
<strong>Exercise 2: Building the model with new chemistry</strong><br><br>
 
Create, configure, build a new case as done in exercise 1, for one of these following configurations: 
- **CAM-chem:** ``f.e21.FCHIST.f09_f09_mg17.tutorial.test2`` with the ``FCHIST`` compset; or 
- **WACCM:** ``f.e21.FWHIST.f09_f09_mg17.tutorial.test2`` with the ``FWHIST`` compset. 

Set up this simulation to run for 5 days, with daily instantaneous output of the variables: ‘PS’,’Z3’,’T’,’U’,’V’,’O3’. 
    
**To change the chemistry**, copy your chemistry preprocessor file chem_mech.in from CaseDocs to your case directory as my_chem_mech.in and change reaction rate for the O1D reaction with O2 to 1.65e-12. Point the simulation to your new mechanism file using ``CAM_CONFIG_OPTS`` and re-build and submit. Once the simulation is complete, compare the 5th day output to the results from exercise 1.


**Extra information:**
    
The chemistry preprocessor generates CAM Fortran source code to solve chemistry. The input is an ASCII file listing chemical reactions and rates. The chemistry preprocessor input file used in your previous run is in your
``$CASEROOT/CaseDocs/chem_mech.in``

An example of a simple chemistry mechanism file content can be seen below:
![chemistry mechanism example](../../../images/challenge/chemistry-mechanism-example.png)

Additional input files for default chemical mechanisms are in each source code subdirectory for mechanisms under:
``$SRCROOT/components/cam/src/chemistry/pp_*``
(i.e. ``pp_waccm_tsmlt_mam4``). You do not need to change these additional files for this exercise.

Below is an overview of the different types of reactions and rates that can be defined in the mechanism file.
![chemistry preprocessor options example](../../../images/challenge/chemistry-preprocessor-info.png)

</div>



<div class="alert alert-warning">  
<details>

<summary> <font face="Times New Roman" color='blue'>Click here for hints</font> </summary>
<br>

**Creating a new chemistry input file**
```    
less CaseDocs/chem_mech.in
cp CaseDocs/chem_mech.in my_chem_mech.in
```
Open your copied new chemistry mechanism file with a text editor and change the reaction rate:
    
![chemistry ozone reaction](../../../images/challenge/chemistry-ozone-reaction.png)

**How to tell CESM to read your new chemistry mechanism file**

- First look at the ``CAM_CONFIG_OPTS`` default setting for CAM-chem <br>
``./xmlquery CAM_CONFIG_OPTS`` <br>
``CAM_CONFIG_OPTS: -phys cam6 -chem trop_strat_mam4_vbs -age_of_air_trcs`` <br>
(note this will be different for WACCM)
    
- Use xmlchange to append a pointer to your user mechanism: <br>
``./xmlchange --append CAM_CONFIG_OPTS="--usr_mech_infile `pwd`/my_chem_mech.in"``
    
- Check the ``CAM_CONFIG_OPTS`` has been changed <br>
``./xmlquery CAM_CONFIG_OPTS`` <br>
``CAM_CONFIG_OPTS: -phys cam6 -chem trop_strat_mam4_vbs -age_of_air_trcs --usr_mech_infile /glade/u/home/$USER/cases/$CASENAME/my_chem_mech.in``
    
- When an important xml variable like CAM_CONFIG_OPTS is changed, resetting the setup is needed. You will need to reset the case setup, and rebuild before submitting. (Note that in general, you only need to reset or clean build when a message shows up in the command prompt stating to reset or clean build).

</details>
</div>



<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font> </summary>
<br>
    
**# Set environment variables** <br>
Use the commands <br>
for **CAM-chem**
```
set CASENAME = f.e21.FCHIST.f09_f09_mg17.tutorial.test2
set CASEDIR = /glade/u/home/$USER/cases/$CASENAME
set RUNDIR = /glade/derecho/scratch/$USER/$CASENAME/run
set COMPSET = FCHIST
set RESOLUTION = f09_f09_mg17
```

or for **WACCM**   
```
set CASENAME = f.e21.FWHIST.f09_f09_mg17.tutorial.test2
set CASEDIR = /glade/u/home/$USER/cases/$CASENAME
set RUNDIR = /glade/derecho/scratch/$USER/$CASENAME/run
set COMPSET = FWHIST
set RESOLUTION = f09_f09_mg17
```

**# Create a new case**

Create a new case with the command ``create_newcase``:
```
cd /glade/u/home/$USER/code/my_cesm_code/cime/scripts/
./create_newcase --case $CASEDIR --res $RESOLUTION --compset $COMPSET
```
You may need to add ``--project "UESM0013"`` when creating a new case.
    
**# Change the job queue and account number**

**If needed**, change ``job queue`` and ``account number``.<br>
For instance, to run in the queue ``tutorial`` and the project number ``UESM0013``
```  
cd $CASEDIR
./xmlchange JOB_QUEUE=tutorial,PROJECT=UESM0013 --force
```
This step can be redone at anytime in the process. 

**# Setup**

From within the case directory, invoke ``case.setup`` with the command:
```    
cd $CASEDIR
./case.setup    
```  
    
**# Build**
    
Perform an initial build within the case directory
```
qcmd -- ./case.build
```
namelists (``atm_in``, ``ice_in``, ``lnd_in``, ``docn_in``) will appear in the CaseDocs
subdirectory, as well as in your $rundir

**# Customize namelists**

As before, edit the file ``user_nl_cam``. First open the ``CaseDocs/atm_in`` and find the lines for avgflag_pertap, mfilt, nhtfrq and copy and paste them into ``user_nl_cam``. Then make changes to those lines in ``user_nl_cam`` and add a line to write out a second history file, ``fincl2``. The namelist lines should look something like:
```
&cam_history_nl
 avgflag_pertape = 'A','I','A','A','A','A','A','A','I'
 mfilt          = 1,30,365,240,240,480,365,73,30
 nhtfrq         = 0,-24,-24,-3,-1,1,-24,-120,-240
 fincl2         = 'PS','Z3','T','U','V','O3'
/
``` 
You can do this with a text editor.

You build the namelists with the command:
```
./preview_namelists
```
This step is optional as the script ``preview_namelists`` is automatically called by ``case.build`` and ``case.submit``. But it is nice to check that your changes made their way into:
```
$CASEDIR/CaseDocs/atm_in
```

**# Change the chemistry**
    
Copy the default mechanism to your own user defined file 
```   
cp CaseDocs/chem_mech.in my_chem_mech.in
```

Open your mechanism file with a text editor (e.g. nedit) and change the reaction rate for O1D with O2
```
nedit my_chem_mech.in &
```
    In CAM-chem
![chemistry ozone reaction](../../../images/challenge/chemistry-ozone-reaction.png)

    In WACCM:
![chemistry ozone reaction](../../../images/challenge/chemistry-ozone-reaction_orig.png)


Append a pointer in the to the user mechanism:
```
./xmlchange --append CAM_CONFIG_OPTS="--usr_mech_infile `pwd`/my_chem_mech.in"
```

**# Re-build the model**
```
./case.setup --reset
./case.build --clean
qcmd -- ./case.build
```
    
**# Set run length**

If needed, change the ``run length``. If you want to run 5 days, you don't have to do this, as 5 days is the default. 
```   
./xmlchange STOP_N=5,STOP_OPTION=ndays
```


**# Submit:**
```
./case.submit
```
------------



**# Check your solution**

When the run is completed, look at the run files and history files.  
    
(1)  Find your model output in your run directory (``$run_dir``) after finished:

```
ls /glade/derecho/scratch/$USER/$CASENAME/run
```
    
(2) Check that your archive directory on derecho (the path will be different on other machines): 
```
cd /glade/derecho/scratch/$USER/archive/$CASENAME/atm/hist
ls 
```

As your run is only 5-day, there should be no monthly file (``h0``)

(3) Look at the contents of the ``h1`` files using ``ncdump``.

```
ncdump –h f.e21.FCHIST.f09_f09_mg17.tutorial.test2.cam.h1.1995-01-01-00000.nc
```

</details>
</div>

